### 1. Fitting the parity function

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

In [4]:
dev = qml.device("default.qubit", wires=4)

In [5]:
def layer(layer_weights):
    for wire in range(4):
        qml.Rot(*layer_weights[wire], wires=wire)

    for wires in ([0, 1], [1, 2], [2, 3], [3, 0]):
        qml.CNOT(wires)

In [6]:
def state_preparation(x):
    qml.BasisState(x, wires=[0, 1, 2, 3])

In [7]:
@qml.qnode(dev)
def circuit(weights, x):
    state_preparation(x)

    for layer_weights in weights:
        layer(layer_weights)
    
    return qml.expval(qml.PauliZ(0))

In [8]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

#### Cost: standard square loss that measures the distance between target labels and model predictions

In [9]:
def square_loss(labels, predictions):
    return np.mean((labels- - qml.math.stack(predictions))**2)

##### Accuracy: the proportion of predictions that agree with a set of target labels

In [10]:
def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc

##### The features and labels considered in the interation of the optimization routine

In [11]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

#### Optimization

In [14]:
data = np.loadtxt("variational_classifier/data/parity_train.txt", dtype=int)
X = np.array(data[:, :-1])
Y = np.array(data[:, -1])
Y = Y * 2 - 1  # shift label from {0, 1} to {-1, 1}

for x,y in zip(X, Y):
    print(f"x = {x}, y = {y}")

x = [0 0 0 1], y = 1
x = [0 0 1 0], y = 1
x = [0 1 0 0], y = 1
x = [0 1 0 1], y = -1
x = [0 1 1 0], y = -1
x = [0 1 1 1], y = 1
x = [1 0 0 0], y = 1
x = [1 0 0 1], y = -1
x = [1 0 1 1], y = 1
x = [1 1 1 1], y = -1


In [21]:
np.random.seed(0)
num_qubits = 4
num_layers = 2
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print("Weights:", weights_init)
print("Bias: ", bias_init)

Weights: [[[ 0.01764052  0.00400157  0.00978738]
  [ 0.02240893  0.01867558 -0.00977278]
  [ 0.00950088 -0.00151357 -0.00103219]
  [ 0.00410599  0.00144044  0.01454274]]

 [[ 0.00761038  0.00121675  0.00443863]
  [ 0.00333674  0.01494079 -0.00205158]
  [ 0.00313068 -0.00854096 -0.0255299 ]
  [ 0.00653619  0.00864436 -0.00742165]]]
Bias:  0.0


In [22]:
opt = NesterovMomentumOptimizer(0.5)
batch_size = 5

In [23]:
weights = weights_init
bias = bias_init
for it in range(100):

    # Update the weights by one optimizer step, using only a limited batch of data
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias = opt.step(cost, weights, bias, X=X_batch, Y=Y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]

    current_cost = cost(weights, bias, X, Y)
    acc = accuracy(Y, predictions)

    print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f}")

Iter:    1 | Cost: 2.3182893 | Accuracy: 0.5000000
Iter:    2 | Cost: 2.3025760 | Accuracy: 0.5000000
Iter:    3 | Cost: 1.7299004 | Accuracy: 0.5000000
Iter:    4 | Cost: 1.5468497 | Accuracy: 0.6000000
Iter:    5 | Cost: 0.9463478 | Accuracy: 0.4000000
Iter:    6 | Cost: 1.0839446 | Accuracy: 0.8000000
Iter:    7 | Cost: 1.7603336 | Accuracy: 0.6000000
Iter:    8 | Cost: 1.1216115 | Accuracy: 0.7000000
Iter:    9 | Cost: 1.1916607 | Accuracy: 0.4000000
Iter:   10 | Cost: 1.1520790 | Accuracy: 0.4000000
Iter:   11 | Cost: 1.7552595 | Accuracy: 0.4000000
Iter:   12 | Cost: 1.0007639 | Accuracy: 0.2000000
Iter:   13 | Cost: 1.3951375 | Accuracy: 0.4000000
Iter:   14 | Cost: 1.6916232 | Accuracy: 0.6000000
Iter:   15 | Cost: 1.0047522 | Accuracy: 0.4000000
Iter:   16 | Cost: 0.8404211 | Accuracy: 0.4000000
Iter:   17 | Cost: 1.8333014 | Accuracy: 0.4000000
Iter:   18 | Cost: 1.1545727 | Accuracy: 0.5000000
Iter:   19 | Cost: 1.1489902 | Accuracy: 0.4000000
Iter:   20 | Cost: 1.4333985 | 

In [24]:
data = np.loadtxt("variational_classifier/data/parity_test.txt", dtype=int)
X_test = np.array(data[:, :-1])
Y_test = np.array(data[:, -1])
Y_test = Y_test * 2 - 1  # shift label from {0, 1} to {-1, 1}

predictions_test = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]

for x,y,p in zip(X_test, Y_test, predictions_test):
    print(f"x = {x}, y = {y}, pred={p}")

acc_test = accuracy(Y_test, predictions_test)
print("Accuracy on unseen data:", acc_test)

x = [0 0 0 0], y = -1, pred=1.0
x = [0 0 1 1], y = -1, pred=1.0
x = [1 0 1 0], y = -1, pred=1.0
x = [1 1 1 0], y = 1, pred=-1.0
x = [1 1 0 0], y = -1, pred=1.0
x = [1 1 0 1], y = 1, pred=-1.0
Accuracy on unseen data: 0.0
